## ChatGPT来了，更快的速度更低的价格

In [1]:
import openai
import os

openai.api_key = os.environ.get("OPENAI_API_KEY")

class Conversation:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})

    def ask(self, question):
        try:
            self.messages.append( {"role": "user", "content": question})
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=self.messages,
                temperature=0.5,
                max_tokens=2048,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e

        message = response["choices"][0]["message"]["content"]
        self.messages.append({"role": "assistant", "content": message})
        
        if len(self.messages) > self.num_of_round*2 + 1:
            del self.messages[1:3]
        return message


In [2]:
prompt = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内"""
conv1 = Conversation(prompt, 3)
question1 = "你是谁？"
print("User : %s" % question1)
print("Assistant : %s\n" % conv1.ask(question1))

question2 = "请问鱼香肉丝怎么做？"
print("User : %s" % question2)
print("Assistant : %s\n" % conv1.ask(question2))

question3 = "那蚝油牛肉呢？"
print("User : %s" % question3)
print("Assistant : %s\n" % conv1.ask(question3))


User : 你是谁？
Assistant : 我是一个人工智能语言模型，专门为您提供中文语言服务。如果您有任何关于做菜的问题需要帮助，我很乐意为您提供帮助。

User : 请问鱼香肉丝怎么做？
Assistant : 鱼香肉丝的做法如下：
1. 瘦肉切成丝，加入盐、淀粉、料酒腌制10分钟；
2. 蒜、姜、葱切末备用；
3. 青红椒切丝备用；
4. 热锅凉油，放入肉丝煸炒至变色，盛出备用；
5. 锅中留底油，放入蒜、姜、葱末煸炒出香味，加入青红椒丝煸炒；
6. 加入肉丝翻炒，加入酱油、糖、醋、盐、味精、水淀粉烧开汁后关火，淋上香油即可。

User : 那蚝油牛肉呢？
Assistant : 蚝油牛肉的做法如下：
1. 牛肉切成薄片，加入盐、淀粉、料酒腌制10分钟；
2. 青红椒、洋葱切成丝备用；
3. 热锅凉油，放入牛肉片煸炒至变色，盛出备用；
4. 锅中留底油，放入青红椒、洋葱丝煸炒；
5. 加入蚝油、酱油、糖、盐、味精、水淀粉烧开汁后放入牛肉片翻炒均匀；
6. 最后淋上少许香油即可上桌。



In [3]:
question4 = "我问你的第一个问题是什么？"
print("User : %s" % question4)
print("Assistant : %s\n" % conv1.ask(question4))


User : 我问你的第一个问题是什么？
Assistant : 你的第一个问题是“你是谁？”。



In [4]:
question5 = "我问你的第一个问题是什么？"
print("User : %s" % question5)
print("Assistant : %s\n" % conv1.ask(question5))

User : 我问你的第一个问题是什么？
Assistant : 你的第一个问题是“请问鱼香肉丝怎么做？”。



### 通过API计算Token数量

In [5]:
class Conversation2:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})

    def ask(self, question):
        try:
            self.messages.append( {"role": "user", "content": question})
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=self.messages,
                temperature=0.5,
                max_tokens=2048,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e

        message = response["choices"][0]["message"]["content"]
        num_of_tokens = response['usage']['total_tokens']
        self.messages.append({"role": "assistant", "content": message})
        
        if len(self.messages) > self.num_of_round*2 + 1:
            del self.messages[1:3]
        return message, num_of_tokens

In [6]:
conv2 = Conversation2(prompt, 3)
questions = [question1, question2, question3, question4, question5]
for question in questions:
    answer, num_of_tokens = conv2.ask(question)
    print("询问 {%s} 消耗的token数量是 : %d" % (question, num_of_tokens))

询问 {你是谁？} 消耗的token数量是 : 118
询问 {请问鱼香肉丝怎么做？} 消耗的token数量是 : 415
询问 {那蚝油牛肉呢？} 消耗的token数量是 : 666
询问 {我问你的第一个问题是什么？} 消耗的token数量是 : 697
询问 {我问你的第一个问题是什么？} 消耗的token数量是 : 690


### 通过Tiktoken库计算Token数量

In [7]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")

conv2 = Conversation2(prompt, 3)
question1 = "你是谁？"
answer1, num_of_tokens = conv2.ask(question1)
print("总共消耗的token数量是 : %d" % (num_of_tokens))

prompt_count = len(encoding.encode(prompt))
question1_count = len(encoding.encode(question1))
answer1_count = len(encoding.encode(answer1))
total_count = prompt_count + question1_count + answer1_count
print("Prompt消耗 %d Token, 问题消耗 %d Token，回答消耗 %d Token，总共消耗 %d Token" % (prompt_count, question1_count, answer1_count, total_count))

总共消耗的token数量是 : 114
Prompt消耗 65 Token, 问题消耗 5 Token，回答消耗 31 Token，总共消耗 101 Token


In [8]:
system_start_count = len(encoding.encode("<|im_start|>system\n"))
print(encoding.encode("<|im_start|>system\n"))
end_count = len(encoding.encode("<|im_end|>\n"))
print(encoding.encode("<|im_end|>\n"))
user_start_count = len(encoding.encode("<|im_start|>user\n"))
print(encoding.encode("<|im_start|>user\n"))
assistant_start_count = len(encoding.encode("<|im_start|>assistant\n"))
print(encoding.encode("<|im_start|>assistant\n"))

total_mark_count = system_start_count + user_start_count + assistant_start_count + end_count*2
print("系统拼接的标记消耗 %d Token" % total_mark_count)


[27, 91, 318, 5011, 91, 29, 9125, 198]
[27, 91, 318, 6345, 91, 397]
[27, 91, 318, 5011, 91, 29, 882, 198]
[27, 91, 318, 5011, 91, 29, 78191, 198]
系统拼接的标记消耗 36 Token


## Gradio帮你快速搭建一个聊天界面

In [9]:
%conda install gradio

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [10]:
import gradio as gr
prompt = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内"""

conv = Conversation(prompt, 5)

def predict(input, history=[]):
    history.append(input)
    response = conv.ask(input)
    history.append(response)
    responses = [(u,b) for u,b in zip(history[::2], history[1::2])]
    return responses, history

with gr.Blocks(css="#chatbot{height:350px} .overflow-y-auto{height:500px}") as demo:
    chatbot = gr.Chatbot(elem_id="chatbot")
    state = gr.State([])

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)

    txt.submit(predict, [txt, state], [chatbot, state])

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
